# Late Arriving Reprocess

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lakelogic/LakeLogic/blob/main/examples/06_advanced_workflows/late_arriving_reprocess/demo_self_healing.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/lakelogic/LakeLogic/blob/main/examples/06_advanced_workflows/late_arriving_reprocess/demo_self_healing.ipynb)

## Business Scenario

Sales data arrives days late, after daily aggregates have already been computed. You need your Gold metrics to heal automatically when late records show up.

## Value Proposition

- Automatically reprocess impacted partitions
- Preserve historical accuracy in Gold metrics
- Avoid full reloads for late data

---

## Goals

1. Run an initial batch
2. Ingest late records
3. Observe self-healing aggregates


##  Step 1: Initial Load (Batch 1)

We process the first batch of events and create our Gold Product.

In [1]:
from pathlib import Path
from lakelogic import DataProcessor
import os
import shutil

def resolve_example_dir(name: str) -> Path:
    cwd = Path.cwd()
    for base in [cwd] + list(cwd.parents):
        if base.name == name and base.exists():
            return base
        candidate = base / name
        if candidate.exists():
            return candidate
        candidate = base / "examples" / "06_advanced_workflows" / name
        if candidate.exists():
            return candidate
        candidate = base / "lakelogic" / "examples" / "06_advanced_workflows" / name
        if candidate.exists():
            return candidate
    return cwd / name

BASE = resolve_example_dir("late_arriving_reprocess")
silver_contract = BASE / "silver_contract.yaml"
gold_contract = BASE / "gold_contract.yaml"

for d in ["data/silver", "data/gold", "logs"]:
    target = BASE / d
    if target.exists():
        shutil.rmtree(target)

silver_proc = DataProcessor(contract=silver_contract)
gold_proc = DataProcessor(contract=gold_contract)

print("Processing Batch 1 (Initial Data)...")
res = silver_proc.run_source(str(BASE / "data" / "raw_sales_batch_1.csv"))
silver_proc.materialize(res.good)

gold_result = gold_proc.run_source()

def preview_frame(df, title=None, limit=5):
    if title:
        print(title)
    if df is None:
        print("<empty>")
        return
    if hasattr(df, "head"):
        try:
            print(df.head(limit))
            return
        except Exception:
            pass
    if hasattr(df, "limit"):
        try:
            print(df.limit(limit))
            return
        except Exception:
            pass
    print(df)

preview_frame(getattr(gold_result, "good", None), title="Initial Gold Revenue Product (sample):")


2026-02-15 12:20:30.702 | INFO     | lakelogic.core.processor:run_source:408 - Loading source: D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\late_arriving_reprocess\data\raw_sales_batch_1.csv via polars


Processing Batch 1 (Initial Data)...


2026-02-15 12:20:31.241 | INFO     | lakelogic.core.processor:run:278 - Starting LakeLogic run [Auto-Engine: polars, Contract: Silver Sales Truth]


2026-02-15 12:20:31.589 | INFO     | lakelogic.core.processor:run:319 - Run complete. Source: 2, Total (post-transform): 2, Good: 2, Quarantined: 0, Pre-Transform Dropped: 0, Ratio: 0.00%


2026-02-15 12:20:38.587 | INFO     | lakelogic.core.materialization:materialize_dataframe:1045 - Materialized 2 rows to partitioned path: D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\late_arriving_reprocess\data\silver\sales


2026-02-15 12:20:38.590 | INFO     | lakelogic.core.processor:run_source:408 - Loading source: data\silver\sales\*\*.parquet via polars


2026-02-15 12:20:38.787 | INFO     | lakelogic.core.processor:run:278 - Starting LakeLogic run [Auto-Engine: polars, Contract: Gold Daily Revenue Product]


2026-02-15 12:20:38.819 | INFO     | lakelogic.core.processor:run:319 - Run complete. Source: 2, Total (post-transform): 2, Good: 2, Quarantined: 0, Pre-Transform Dropped: 0, Ratio: 0.00%


Initial Gold Revenue Product (sample):
shape: (2, 4)
┌─────────────────────┬──────────┬───────────────┬─────────────┐
│ event_date          ┆ store_id ┆ total_revenue ┆ event_count │
│ ---                 ┆ ---      ┆ ---           ┆ ---         │
│ datetime[ms]        ┆ str      ┆ f64           ┆ u32         │
╞═════════════════════╪══════════╪═══════════════╪═════════════╡
│ 2024-02-06 00:00:00 ┆ STORE_A  ┆ 150.0         ┆ 1           │
│ 2024-02-05 00:00:00 ┆ STORE_A  ┆ 100.0         ┆ 1           │
└─────────────────────┴──────────┴───────────────┴─────────────┘


##  Step 2: The Late Arrival (Batch 2)

Now a record for **2024-02-05** (Monday) arrives late. Notice that we don't have to change any codewe just run the same contracts.

In [2]:
print("Processing Batch 2 (Late Arrival for Feb 5th)...")
res = silver_proc.run_source(str(BASE / "data" / "raw_sales_batch_2_LATE.csv"))
silver_proc.materialize(res.good)

print("Gold layer detecting changes and healing product...")
gold_result_healed = gold_proc.run_source()

preview_frame(getattr(gold_result_healed, "good", None), title="Healed Gold Revenue Product (sample):")


2026-02-15 12:20:38.861 | INFO     | lakelogic.core.processor:run_source:408 - Loading source: D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\late_arriving_reprocess\data\raw_sales_batch_2_LATE.csv via polars


2026-02-15 12:20:38.906 | INFO     | lakelogic.core.processor:run:278 - Starting LakeLogic run [Auto-Engine: polars, Contract: Silver Sales Truth]


2026-02-15 12:20:38.936 | INFO     | lakelogic.core.processor:run:319 - Run complete. Source: 1, Total (post-transform): 1, Good: 1, Quarantined: 0, Pre-Transform Dropped: 0, Ratio: 0.00%


2026-02-15 12:20:38.970 | INFO     | lakelogic.core.materialization:materialize_dataframe:1045 - Materialized 1 rows to partitioned path: D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\late_arriving_reprocess\data\silver\sales


2026-02-15 12:20:38.973 | INFO     | lakelogic.core.processor:run_source:408 - Loading source: data\silver\sales\*\*.parquet via polars


Processing Batch 2 (Late Arrival for Feb 5th)...
Gold layer detecting changes and healing product...


2026-02-15 12:20:39.037 | INFO     | lakelogic.core.processor:run:278 - Starting LakeLogic run [Auto-Engine: polars, Contract: Gold Daily Revenue Product]


2026-02-15 12:20:39.051 | INFO     | lakelogic.core.processor:run:319 - Run complete. Source: 2, Total (post-transform): 2, Good: 2, Quarantined: 0, Pre-Transform Dropped: 0, Ratio: 0.00%


Healed Gold Revenue Product (sample):
shape: (2, 4)
┌─────────────────────┬──────────┬───────────────┬─────────────┐
│ event_date          ┆ store_id ┆ total_revenue ┆ event_count │
│ ---                 ┆ ---      ┆ ---           ┆ ---         │
│ datetime[ms]        ┆ str      ┆ f64           ┆ u32         │
╞═════════════════════╪══════════╪═══════════════╪═════════════╡
│ 2024-02-06 00:00:00 ┆ STORE_A  ┆ 150.0         ┆ 1           │
│ 2024-02-05 00:00:00 ┆ STORE_A  ┆ 50.0          ┆ 1           │
└─────────────────────┴──────────┴───────────────┴─────────────┘


##  Why did this work?

1.  **Lineage Watermark**: The late record in Silver was tagged with a `_lakelogic_processed_at` time of **Now**.
2.  **Incremental Selection**: Gold saw that new records were added to Silver since the last run.
3.  **Partition Healing**: Because we used `reprocess_policy: overwrite_partition_safe`, LakeLogic calculated the new aggregate for Feb 5th and replaced the old file in the `event_date=2024-02-05` directory.

This is the heart of **Data as a Product**: No manual maintenance, just a self-healing system.